In [ ]:
using Plots
using LaTeXStrings
using LinearAlgebra
using ForwardDiff

# Introdução ao problema de Interpolação (polinomial)

Em algumas situações é possível que conheçamos uma função $f: [a, b] \rightarrow
\mathbb{R}$ em alguns pontos apenas. Isso pode ocorrer, por exemplo, porque o
custo para computá-la é muito elevado. Como quando a função é resultado de uma
sequência longa de operações computacionais envolvendo simulações de fenômenos
complexos. Ou quando a função é obtida experimentalmente. Uma pergunta natural é
se é possível, de alguma forma razoável, extrapolar essa informação para todo o
intervalo de interesse.

Infelizmente esse problema é muito vago. De fato considere o problema de
aproximar a função seno conhecendo os seus valores nos ângulos notáveis.

| $x$       | $0$ | $\frac{\pi}{4}$      | $\frac{\pi}{3}$      | $\frac{\pi}{2}$ |
| --------- | --- | -------------------- | -------------------- | --------------- |
| $\sin(x)$ | $0$ | $\frac{\sqrt{2}}{2}$ | $\frac{\sqrt{3}}{2}$ | $1$             |

Veja o gráfico da função abaixo com os pontos tabelados destacados.

In [ ]:
# Dados
angulos = [0, π/4, π/3, π/2]

# Plota o gráfico de seno usando 1000 pontos.
x = range(0, π/2, length = 100)
scatter(angulos, sin.(angulos), color = :red, label = "", ratio = 1)
plot!(x, sin.(x), linestyle = :dash, color = :black, label = "")

Nesse caso é claro que, além da função seno, há múltiplas (infinitas) funções,
contínuas ou não, que passam pelos pontos vermelhos. Veja outro exemplo:


In [ ]:
function f(x)
    # Depois de ler o texto todo volte aqui e pense como gerei essa
    # função.
    num1 = sin(π/4)*cos(x - π/2)*cos(x + π/6)*cos(x)  
    den1 = cos(π/4 - π/2)*cos(π/4 + π/6)*cos(π/4)
    num2 = sin(π/3)*cos(x - π/2)*cos(x + π/4)*cos(x) 
    den2 = cos(π/3 - π/2)*cos(π/3 + π/4)*cos(π/3)
    num3 = sin(π/2)*x*(x - π/4)*(x - π/3) 
    den3 = π/2*π/4*π/6 
    return num1 / den1 + num2 / den2 + num3 / den3
end

plot!(x, f.(x), color = :blue, label = "")

O nosso desejo, entretanto, é buscar uma função que seja "bem comportada", que
possua expressão simples, que não possua variações bruscas.

Uma das maneiras de controlar a complexidade das funções que podemos usar na
aproximação é definir uma classe de funções de trabalho, que sejam bem
conhecidas e possuam boas propriedades. Um das classes de funções bem
comportadas e que são fáceis de calcular são as funções polinomiais. Isso nos
leva ao problema.

# Definição formal da interpolação polinomial e uma primeira solução

Dado um conjunto de pares 
$(x_i, y_i) \in \mathbb{R} \times \mathbb{R}, i = 0, \ldots, n$, 
com as abcissas todas distintas, desejamos encontrar um polinômio
$p$, tal que $p(x_i) = y_i,\ i = 0, \ldots, n$.

Esse problema é bem mais simples e bem definido do que o problema aberto de
encontrar uma função qualquer que passa pelos pontos. Para entender como
resolvê-la vamos começar com um exemplo com números simples e tentar resolvê-lo.
Vamos tentar achar um polinômio coincida com os valores dados na tabela

| $x$ | -1 | 2 | 3 |
|-----|----|---|---|
| $y$ | 6  | 3 |10 |

Uma primeira pergunta é qual é o grau do polinômio desejado. Como temos o valor
do polinômio em três pontos, é natural considerar um polinômio de grau 2, que
possui três coeficientes a determinar. Ou seja um polinômio na forma
$$
p(x) = c_0 + c_1 x + c_2 x^2
$$
Agora, dizer que esse polinômio passa pelos pontos na tabela é dizer que
$$
\begin{matrix}
c_0 &+& c_1 (-1) &+& c_2 (-1)^2 &=& p(-1) &=& 6\\
c_0 &+& c_1 2 &+& c_2 2^2 &=& p(2) &=& 3 \\
c_0 &+& c_1 3 &+& c_2 3^2 &=& p(3) &=& 10.
\end{matrix}
$$
Note que ocorreu algo análogo ao que ocorria no problema de quadrados mínimos.
Recaímos em um sistema linear.
$$\left\{
\begin{matrix}
c_0 & - & c_1   & + & c_2   &=& 6\\
c_0 & + & 2 c_1 & + & 4 c_2 &=& 3 \\
c_0 & + & 3 c_1 & + & 9 c_2 &=& 10.
\end{matrix}
\right.$$
De fato, podemos esquematizar o sistema com os dados da tabela vendo de maneira
ainda mais clara de onde ele veio. Veja que surge a mesma matriz que usamos para
definir o sistema normal no caso de quadrados mínimos.
$$\begin{bmatrix}
1 & x_1 & x_1^2 \\
1 & x_2 & x_2^2 \\
1 & x_3 & x_3^2
\end{bmatrix} \begin{bmatrix}
c_0 \\
c_1 \\
c_2
\end{bmatrix} = \begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}.$$
Substituindo os dados da tabela obtemos o sistema acima.

Resolvendo o sistema obtermos $p(x) = 1 - 3x + 2x^2$. E podemos testar se esse
polinômio de fato passa pelos pontos da tabela

In [ ]:
# Tabela de dados
xs = [-1, 2, 3]
ys = [6, 3, 10]
n = length(xs)

# Encontra o polinômio interpolador usando a matriz de Vandermond
A = [ones(n) xs xs.^2]
# Resolve o sistema A coefs = ys
coefs = A \ ys 
# Definie o polinômio, note que vetores de Julia começam no 1
p(x) = coefs[1] + coefs[2]*x + coefs[3]*x^2
@show coefs

# Desenha os dados da tabela e o polinômio
xxs = range(xs[1], xs[end], length = 100)
scatter(xs, ys, color = :red, label = "")
plot!(xxs, p.(xxs), color = :blue, label = L"p(x)", legend = :topleft, ratio = 1)

Perfeito!

Observem que as ideias que nos levaram a resolver esse caso podem ser
naturalmente estendidas para lidar com o caso geral de $n + 1$ pontos na tabela. 
Vamos fazer isso. Considere a tabela

| $x$ | $x_0$ | $x_2$ | $\ldots$ | $x_n$ |
|-----|-------|-------|----------|--------
| $y$ | $y_0$ | $y_2$ | $\ldots$ | $y_n$ |


Podemos tentar encontrar um polinômio de grau $n$  na forma
$$p(x) = c_0 + c_1 x + c_2 x^2 + \ldots + c_{n} x^{n}$$
tal que $p(x_i) = y_i,\ i = 1, 2, \ldots, n$. Para isso escremos o sistema de
equações
$$\begin{split}
c_0 + c_1 x_0 + c_2 x_0^2 + \ldots + c_{n} x_0^{n} &= p(x_0) = y_0\\
c_0 + c_1 x_1 + c_1 x_0^2 + \ldots + c_{n} x_1^{n} &= p(x_1) = y_1\\
\vdots \hspace{3cm} & \hspace{1cm} \vdots\\
c_0 + c_1 x_n + c_2 x_n^2 + \ldots + c_{n} x_n^{n} &= p(x_n) = y_n.\\
\end{split}$$
Um sistema linear com  e $n + 1$ equações e $n$ variáveis, os coeficientes 
$c_0, c_1, c_2, \ldots, c_{n}$.

Alternativamente, esse mesmo sistema linear pode se escrito usando a matriz de
Vandermonde
$$\begin{bmatrix}
1 & x_0 & x_0^2 & \ldots & x_0^{n} \\
1 & x_1 & x_1^2 & \ldots & x_1^{n} \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
1 & x_n & x_n^2 & \ldots & x_n^{n} \\
\end{bmatrix} \begin{bmatrix}
c_0 \\
c_1 \\
\vdots \\
c_n
\end{bmatrix} = \begin{bmatrix}
y_0 \\
y_1 \\
\vdots \\
y_n
\end{bmatrix}.$$

Notem que há uma equivalência completa entre o problema de interpolação
polinomial e esse sistema. Se o sistema tiver uma solução, essa solução resolve
o problema de interpolação polinomial e vice versa. 

Resta ainda definir condições que digam quando o sistema acima tem soluções.
Outra pergunta relevante é se a solução será única ou se é possível que existam
múltiplas soluções. Do nosso estudo de álgebra linear, sabemos que a resposta
para essa perguntas estão na matriz de Vandermonde. O problema terá solução
única sempre que a matriz for não singular, ou seja, quando o seu determinante
for não nulo.

## Polinômios de Lagrange

Vamos analisar o problema de exitência e unicidade de soluções por um
outro caminho que irá deixar mais fácil o processo de solução, evitando a
necessidade de se resolver sistemas lineares. Para isso vamos retomar o exemplo
original, baseado na tabela

| $x$ | -1 | 2 | 3 |
|-----|----|---|---|
| $y$ | 6  | 3 |10 |

Ao resolvermos esse problemas tentamos encontrar um polinômio na forma
$$p(x) = c_0 + c_1 x + c_2 x^2,$$
ou seja, fixamos uma base do espaço dos polinômios de grau até 2, $1, x, x^2$ e
buscamos uma combinação linear dos elementos da base para definir o polinômio.
Porque escolher essa base? É a única opção? Não há outras bases mais
interessantes? 

Vamos fazer uma modificação simples na base e ver o que ocorre. Vamos considerar
que queremos escrever o polinômio na forma
$$p(x) = \alpha + \beta (x - 3) + \gamma (x - 3)^2.$$
Claramente qualquer polinômio de grau até 2 pode se escrito nessa forma. Não
estamos perdendo possíveis soluções.

O sistema que temos que resolver, com os coeficientes $\alpha, \beta$ e $\gamma$
como variáveis é
$$\begin{array}{lllllll}
\alpha &+& \beta (-1 - 3) &+& \gamma (-1 - 3)^2 &=& p(-1) = 6\\
\alpha &+& \beta (2 - 3)  &+& \gamma (2 - 3)^2  &=& p(2) = 3 \\
\alpha &+& \beta (3 - 3) &+& \gamma (3 - 3)^2 &=& p(3) = 10.
\end{array}$$
Note que os valores $3 - 3$ se anulam, reduzindo o sistema a
$$\begin{matrix}
\alpha &-& 4 \beta &+& 16 \gamma &=& 6\\
\alpha &-& \beta &+& \gamma &=& 3 \\
\alpha &&&& &=& 10.
\end{matrix}$$
Veja que a vida ficou mais fácil. A última equação já diz o valor da $\alpha$ e
ficamos apenas com um sistema de duas equações e duas variáveis para resolver
$$\begin{matrix}
- \beta &+& 4 \gamma &=& -1\\
- \beta &+& \gamma  &=& -7 \\
\end{matrix}$$
Esse sistema tem solução $\beta = 9, \gamma = 2$ (verifique). Assim o polinômio
procurado é
$$p(x) = 10 + 9(x - 3) + 2 (x - 3)^2.$$
Observe que esse polinômio é o mesmo encontrado antes, só que escrito de outra
forma (mais uma vez verifique). Mas não vamos focar nessa questão de unicidade
por enquanto. Vamos tentar entender porque o sistema ficou mais simples.

O segredo foi escolher como base do espaço de polinômios $1, x - 3, (x - 3)^2$
polinômios que se anulam em uma das abicissas $x_2 = 3$. Nesse caso, dois dos
polinômios da base valem zero e isso se reflete no sistema com uma equações mais
simples quando estamos escrevendo $p(x_2) = y_2$. Nesse caso a equação ficou
tão simples que o valor do coeficiente associdao ao polinômio constante $1$
ficou explícito e isso diminuiu o número de variáveis.

Essa parece uma boa ideia. Será que conseguimos ir mais para frente? Será que
conseguimos escolher polinômios que tem a propriedade que eles se anulam na
maior parte dos pontos da tabela? Mais uma vez vamos fazer isso para a nossa
tabela com apenas 3 pontos e depois tentar generalizar para $n$ pontos. 

No exemplo acima, ao escolhermos 3 polinômios no qual dois deles se anulavam no
ponto $x_3$ deixou a equação associada à condição $p(x_2) = y_2$ simples. Vamos
tentar avançar mais nessa ideia. Será que conseguimos três polinômios de grau
até 2, $p_0, p_1, p_2$ tais que $p_1$ e $p_2$ se anulem no $x_0$, de modo a
simplificar a equação associada a $p(x_0) = y_0$, $p_0$ e $p_2$ se anulem no
$x_1$ e $p_0$ e $p_1$ se anulem no $x_2$? Parece complicado, não? 

Esse é o tipo do problema no estilo "ovo de Colombo", ao vermos a solução
dizemos para nós mesmos: "mas era tão simples..." Vejamos polinômios que
funcionam:
$$\begin{array}{ll}
p_0(x) = (x - x_1)(x - x_1) &\quad\quad \text{[Se anula em $x_1$ e $x_2$]} \\
p_1(x) = (x - x_0)(x - x_2) &\quad\quad \text{[Se anula em $x_0$ e $x_2$]} \\
p_2(x) = (x - x_0)(x - x_1) &\quad\quad \text{[Se anula em $x_0$ e $x_1$]}
\end{array}$$
Aí estão exemplos dos polinômios desejados.

Agora que temos três polinômios (de grau até 2) com propriedades que vão
facilitar as contas, vamos tentar resolver o problema de interpolação com um
polinômio geral $p$ que é combinação linear desses
$$p(x) = c_0 p_0(x) + c_1 p_1(x) + c_2 p_2(x).$$
As condições de interpolação para a nossa velha amiga

| $x$ | -1 | 2 | 3 |
|-----|----|---|---|
| $y$ | 6  | 3 |10 |

ficam:
$$\begin{matrix}
c_0 p_0(-1) &+& c_1 p_1(-1) &+& c_2 p_2(-1) &=& p(-1) &=&  6 \\
c_0 p_0(2) &+& c_1 p_1(2) &+& c_2 p_2(2) &=& p(2) &=&  3 \\
c_0 p_0(3) &+& c_1 p_1(3) &+& c_2 p_2(3) &=& p(3) &=&  10.
\end{matrix}$$
Lembrando que os polinômios $p_i$ são feitos para se anular em $x_j, j \neq i$,
o sistema simplifica para
$$\begin{split}
c_0 p_0(-1) &=  6 \\
c_1 p_1(2)  &=  3 \\
c_2 p_2(3)  &=  10.
\end{split}$$
Ou, em outras palavras:
$$\begin{split}
c_0 &=  6 / p_0(-1) \\
c_1 &=  3 / p_1(2) \\
c_2 &=  10 / p_2(3).
\end{split}$$
Os coeficientes desejados são obtidos trivialmente.

Será que podemos deixar isso ainda mais fácil? Que tal procurar um polinômio
que, como $p_0$ se anula em $x_1$ e $x_2$ e que também vale $1$ em $x_0$?
Pensando um pouco, chegamos em
$$l^2_0(x) = \frac{(x - x_1)(x - x_2)}{(x_0 - x_1)(x_0 - x_2)}.$$
Este polinômio está bem definido porque as abcissas $x_0$, $x_1$, $x_2$ são
todas distintas. De maneira análoga podemos chegar em variações de $p_1$ que
vale 1 em $x_1$ e de $p_2$ que vale $1$ em $x_2$.
$$l^2_1(x) = \frac{(x - x_0)(x - x_2)}{(x_1 - x_0)(x_1 - x_2)} \quad
\text{e}\quad l^2_2(x) = \frac{(x - x_2)(x - x_1)}{(x_2 - x_0)(x_2 - x_1)}.$$
Vejamos esses polinômios.

In [ ]:
# Lembre que vetore em Julia começam no 1 e não no 0
l20(x) = (x - xs[2])*(x - xs[3]) / (xs[1] - xs[2]) / (xs[1] - xs[3])
l21(x) = (x - xs[1])*(x - xs[3]) / (xs[2] - xs[1]) / (xs[2] - xs[3])
l22(x) = (x - xs[1])*(x - xs[2]) / (xs[3] - xs[1]) / (xs[3] - xs[2])
plot(xxs, l20.(xxs), lw = 2, label = L"l^2_0", ratio = 1)
plot!(xxs, l21.(xxs), lw = 2, label = L"l^2_1")
plot!(xxs, l22.(xxs), lw = 2, label=L"l^2_2")
scatter!([xs xs], [0 0 0 1 1 1], color = :black, label = "")

Esses polinômios especiais são conhecidos como polinômios de Lagrange. Note que
se escrevermos as condições de interpolação do exemplo, partindo de um polinômio
na forma 
$$p(x) = c_0 l^2_0(x) + c_1 l^2_1(x) + c_2 l^2_2(x)$$ 
chegamos às equações
$$\begin{split}
c_0 l^2_0(-1) &=  6 \\
c_1 l^2_1(2)  &=  3 \\
c_2 l^2_2(3)  &=  10.
\end{split}$$
Como os polinômios $l^2_i$ valem $1$ em $x_i$, isso é ainda mais simples:
$$\begin{split}
c_0 &=  6 \\
c_1 &=  3 \\
c_2 &=  10.
\end{split}$$
Os coeficientes são os próprios valores presentes na linha $y$ da tabela.

Como generalizar isso? Não é muito difícil, o mais importante é usar a notação
certa para obter expressões simples e compactas. No caso geral recebemos uma
tabela para interpolar _com as abicissas todas distintas_:

| $x$ | $x_0$ | $x_1$ | $\ldots$ | $x_n$ |
|-----|-------|-------|----------|--------
| $y$ | $y_0$ | $y_1$ | $\ldots$ | $y_n$ |

Análogo ao que foi feito para o caso particular, queremos definir polinômios
$l^{n}_i(x), i = 0, 1, ..., n$ tais que:
1. $l^{n}_i(x_j) = 0, j \neq i$
2. $l^{n}_i(x_i) = 1$.

Penando mais um pouco a fórmula geral para os poliômios de Lagrange de grau $n$
é
$$l^n_i(x) = \prod_{\substack{j = 0\\ j \neq i}}^n \frac{x - x_j}{x_i - x_j},\ i
= 0, 1, \ldots, n.$$
Já a fórmula para o polinômio interpolador é
$$p(x) = \sum_{i = 0}^n y_i l^n_i(x).$$

Em particular provamos que a solução do problema de interpolação sempre existe.
Não importa qual seja o lado direito (ou seja o vetor $y$). Isso mostra que
matriz de Vandermonde é não singular. Mostrando que a solução deve ser única
também.

Podemos confirmar isso de outra forma. Considere duas possíveis soluções do
problema de interpolar uma tabela com $n + 1$ entradas, com abicissas todas
distintas, por um polinômio de grau até $n$. Chamemos essas duas soluções de
$p_1$ e $p_2$. Vamos definir o polinômio $q(x) = p_1(x) - p_2(x)$. O que sabemos
sobre ele? Primeiro $q$ é um polinômio de grau até $n$, já que assim são $p_1$ e
$p_2$. Além disso sabemos que
$$q(x_i) = p_1(x_i) - p_2(x_i) = y_i - y_i = 0,\ i = 0, 1, \ldots n.$$
Ou seja, $q$ é um polinômio de grau até $n$ com $n + 1$ raízes distintas! Pelo
teorema fundamental da álgebra $q$ deve ser então o polinômio constante nulo.
Ou, seja, vemos que 
$$ \forall x,\ 0 = q(x) = p_1(x) - p_2(x) \implies \forall x,\ p_1(x) = p_2(x). $$
Ou seja, as duas soluções são a mesma! Note que isso é válido, não importanto
como obtivemos $p_1$ e $p_2$. Um pode ter sido obtido a partir da matriz de
Vandermonde e o outro a partir de polinômios de Lagrange. 

Coroamos toda essa discussão com o seguinte teorema

**Teorema.** Sejam $x_0, x_1, \ldots, x_n \in \mathbb{R}$ $n + 1$ valores todos
distintos e $y_0, y_1, \ldots, y_n \in \mathbb{R}$. Então existe um único
polinômio (interpolador) $p$ tal que
$$p(x_i) = y_i,\ i = 0, 1, \ldots, n.$$
Esse polinômio pode ser obtido resolvendo o sistema baseado na matriz de
Vandermonde ou através dos polinômios de Lagrange. 

# Variações do problema de interpolação

O problema de interpolação polinomial pode aparecer de outras formas na literatura além de pontos e valores funcionais. Uma variante comum e bastante útil é o problema de encontrar um polinômio que passe por alguns valores tabelados, mas também com suas derivadas tabeladas. Por sorte derivadas de polinômios são também polinômios com coeficientes herdados do polinômio original. Assim esse problema tem as mesmas características do problema original e pode ser atacado de forma semelhante. Vamos ver isso, inicialmente, através de um exemplo.

#### Exercício

Dada a tabela

| $x$ |  1 |  2 |
|-----|----|----|
| $y$ |  1 | -2 |
| $y'$| -1 | -7 |

Encontre um polinômio interpolador.

Mais uma vez, é natural pensarmos em usar um polinômio de grau 3, pois mais uma vez teremos quatro coeficientes para determinar e temos quatro condições. Aa fórmulas gerais de um polinômio e sua derivada assim são:
$$
p(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3,\quad p'(x) = c_1 + 2 c_2 x + 3 c_3 x^2.
$$
Deste modo, as condições descritas pela tabela são
$$\begin{matrix}
c_0 + c_1 1 + c_2 1^2 + c_3 1^3 &=& c_0 + c_1 + c_2 + c_3 &=& p(1) &=& 1 \\
c_0 + c_1 2 + c_2 2^2 + c_3 2^3 &=& c_0 + 2 c_1 + 4 c_2 + 8 c_3 &=& p(2) &=& -2 \\
c_1 + 2 c_2 1 + 3 c_3 1^2 &=& c_1 + 2 c_2 + 3 c_3 &=& p'(1) &=& -1 \\
c_1 + 2 c_2 2 + 3 c_3 2^2 &=& c_1 + 4 c_2 + 12 c_3 &=& p'(1) &=& -7.
\end{matrix}$$
Isso pode ser, facilmente, resolvido no computador por 

In [ ]:
A = [1 1 1 1
     1 2 4 8
     0 1 2 3
     0 1 4 12]
b = [1, -2, -1, -7]
c = A \ b # Resolve o sistema Ac = b

Pronto, o polinômio interpolador é

In [ ]:
p(x) = c[1] +c[2]*x + c[3] * x^2 + c[4]*x^3
p′(x) = c[2] + 2*c[3]*x + 3*c[4]*x^2
@show p(1), p(2)
@show p′(1), p′(2);

Note que no exemplo acima ainda caimos na situação de resolver um sistema linear. A pergunta natural então passa a ser: tem como evitar isso? Há algo como polinômios de Lagrange para esse tipo de problema? Para isso vejamos um outro exemplo.

#### Exercício

Encontre um polinômio de grau 3 tal que
$$\begin{split}
p(-1) = \alpha_0, &\quad p(1) = \alpha_1 \\
p'(-1) = \beta_0, &\quad p'(1) = \beta_1.
\end{split}$$
Para isso considere polinômios que são combinações lineares de
$$\frac{(x + 2)(x - 1)^2}{4}, \ 1 - \frac{(x + 2)(x - 1)^2}{4}, \ \frac{(x + 1)(x - 1)^2}{4}, \ \frac{(x + 1)^2(x - 1)}{4}.$$

Vamos começar resolvendo o exerício apresentando p gráfico dessas funções, para ver porque foram escolhidas:

In [ ]:
p1(x) = (x + 2)*(x - 1)^2 / 4
p2(x) = 1 - (x + 2)*(x - 1)^2 / 4
p3(x) = (x + 1)*(x - 1)^2 / 4
p4(x) = (x + 1)^2*(x - 1) / 4
xs = range(-2, 2, length=200)
plot(xs, p1.(xs), lw = 2, label = L"p_1")
plot!(xs, p2.(xs), lw = 2, label = L"p_2")
plot!(xs, p3.(xs), lw = 2, label = L"p_3")
plot!(xs, p4.(xs), lw = 2, label = L"p_4")

Bingo! Esses polinômios desempenham um papel semelhante aos polinômios de Lagrange. Por exemplo, podemos ver que o primeiro, $p_1$ é tal que
$$p_1(1) = 1,\ p_1'(-1) = 0,\ p_1(1) = 0,\ p_1'(1) = 0.$$ 
E relações semelhantes são obedecidas pelos outros polinômios. 

Isso sugere que a solução é
$$p(x) = \alpha_0 p_1(x) + \alpha_1 p_2(x) + \beta_0 p_3(x) + \beta_1 p_4(x)$$
Vamos verificar isso numericamente em um caso particular?

In [ ]:
α₀, α₁, β₀, β₁ = 1, 2, π, √2
p(x) = α₀*p1(x) + α₁*p2(x) + β₀*p3(x) + β₁*p4(x)
@show p(-1), p(1)
@show ForwardDiff.derivative(p, -1), ForwardDiff.derivative(p, 1);

Isso nos ensina que é possível criar, para diferentes variações do problema de interpolação polinômial, diferentes grupos de polinômio que obedecem as seguintes propriedades:

* Todas as condições valem zero neles menos 1.
* Nesse, a condição vale 1.

Com isso, a reolução do problema de interpolação polinomial é simples, basta pegar os valores tabelados.

Agora que já vimos um pouco sobre como resolver o problema de interpolação polinomial. Vamos ver se os resultados que obtemos são úteis. Ou seja, será que usar o polinômio no lugar faz sentido? Será que ajuda? Para isso vamos retomar o exemplo inicial, da função seno. Lembrando a nossa tabela era

| $x$       | $0$ | $\frac{\pi}{4}$      | $\frac{\pi}{3}$      | $\frac{\pi}{2}$ |
| --------- | --- | -------------------- | -------------------- | --------------- |
| $\sin(x)$ | $0$ | $\frac{\sqrt{2}}{2}$ | $\frac{\sqrt{3}}{2}$ | $1$             |

Vamos resolver o problema de interpolação e comparar com os valores reais da função.

In [ ]:
# Resolve o problema de interpolacao e testa em um novo ponto dentro do intervalo
xs = [0, π/4, π/3, π/2]
ys = [0, √2 / 2, √3 /2, 1]
A = [ones(size(xs)) xs xs.^2 xs.^3]
coefs = A \ ys
p(x) = coefs[1] + coefs[2]*x + coefs[3]*x^2 + coefs[4]*x^3
@show sin(1), p(1);

Nada mal, o polinômio acerta o seno de 1 até a terceira casa. Vamos coparar os gráficos.

In [ ]:
xxs = range(0, π / 2, length=200)
plot(xxs, sin.(xxs), lw=2, label = "seno")
plot!(xxs, p.(xxs), lw=8, color=1, alpha = 0.5, label = "polinômio", legend = :topleft)

Visuamente são quase as mesmas funções! Dá para notar que o seno real não fica sempre no meio da faixa, ou seja há algum erro entre elas, mas a precisão já é bastante interessante.

Agora o que ocorre fora do intervalo de interpolação?

In [ ]:
xxs = range(-π / 2, π, length=400)
plot(xxs, sin.(xxs), lw=2, label = "seno")
plot!(xxs, p.(xxs), lw=8, color=1, alpha = 0.5, label = "polinômio", legend = :topleft)

Ops... Fora do intervalo de interpolação não há grande coincidência. E a coisa fica, obviamente ainda mais dramática de aumentarmos ainda mais a faixa.

In [ ]:
xxs = range(-5, 5, length=400)
plot(xxs, sin.(xxs), lw=2, label = "seno")
plot!(xxs, p.(xxs), lw=8, color=1, alpha = 0.5, label = "polinômio", legend = :topright)

# Implementação de polimônios de Lagrage

A fórmula dos polinômios de Lagrange
$$l^n_i(x) = \prod_{\substack{j = 0\\ j \neq i}}^n \frac{x - x_j}{x_i - x_j},\ i
= 0, 1, \ldots, n.$$
pode levar a uma implementação ineficiente e com problemas numéricos. Por exemplo você pode pensar que para avaliar cada um deles será necessário calcular $n$ produtos e somas. Assim, para avaliar o polinômio interpolador, que envolve todos os $n + 1$ polinômios de Lagrange, em um novo ponto $x$ seria necessário algo da ordem $n^2$ para calculá-los em um novo $x$.

Mas é possível, pensando um pouco, evitar muitas desses contas e problemas. Essa é a ideia por detrás de _interpolação baricêntrica de Lagrange_, um nome complicado para uma ideia simples.

Inicialmente vamos definir um polinômio
$$l(x) = \prod_{j = 0}^n (x - x_j).$$
Ou seja, ele usa todos os termos. Além disso vamos definir $n + 1$ valores constantes:
$$w_i = \frac{1}{\prod_{\substack{j = 0\\ j \neq i}}^n (x_i - x_j)},\quad i = 0, 1, 2, \ldots, n.$$
Veja que todos esses valores, $w_i,\ i = 0, 1, \ldots, n$ só dependem dos dados usados na interpolação e, portanto, podem ser calculados uma única vez.

Segue que
$$
l^n_i(x) = l(x) \frac{w_i}{x - x_i}.
$$
Já o polinômio interpolador ficar
$$
\begin{split}
p(x) &= y_0 l^n_0(x) + y_1 l^n_1(x) + \ldots + y_n l^n_n(x) \\
&=  y_0 l(x) \frac{w_0}{x - x_0} + y_1 l(x) \frac{w_1}{x - x_1} + \ldots + y_n l(x) \frac{w_n}{x - x_n}. \\
&= l(x) \left( y_0 \frac{w_0}{x - x_0} + y_1 \frac{w_1}{x - x_1} + \ldots + y_n \frac{w_n}{x - x_n} \right).
\end{split}
$$
Ou seja, para calcular o polinômio interpolador, basta calular $l(x)$, com $n$ somas e produtos e depois relizar mais um número proporcional a $n$ de operações. 


In [ ]:
# Colocar uma implementação dessa ideia aqui

# Qualidade da interpolação

Como vimos, a motivação da interpolação é conseguir uma aproximação da função nos pontos não tabelados. Uma pergunta fundamental nesse contexto é qual é a qualidade dessa aproximação. O objetivo dessa seção é responder essa pergunta mesmo que parcialmente. Para isso vamos fazer algumas hipóteses.

#### Hipótese

* A função que está sendo interpolada $f$ possui $n + 1$ derivadas contínuas.
* O polinômio interpolador tem grau no máximo $n$.
* $p$ interpola $f$ em $\{x_0, x_1, \ldots, x_n \} \subset [a b]$.

Nosso objetivo é estudar qual é o erro de interpolação
$$E(x) := f(x) - p(x)$$
sob essas hipóteses para $x \in (a, b)$ fixo, com $x \neq x_i,\ i = 0, 1, \ldots, n$.
Para isso, vamos considerar a função auxiliar
$$g(z) = E(x)\omega(z) - E(z)\omega(x),$$
em que $\omega(z) = (z - x_0)(z - x_1)\ldots(z - x_n)$. 

Essa função possui possui pelo menos $n + 1$ derivadas contínuas (herdadas de $f$). Outra característica interessante é que
$$g(x_i) = E(x)\omega(x_i) - E(x_i)\omega(x) = E(x) \cdot 0 - 0 \cdot \omega(x) = 0.$$
Além disso,
$$g(x) = E(x)\omega(x) - E(x)\omega(x) = 0.$$
Ou seja, $g$ tem pelo menos $n + 2$ raízes distintas. 

Vamos agora relembrar um resultado bem conhecido de cálculo

#### Teorema de Rolle.

Seja $f$ uma função diferenciável em $[a, b]$. Se $f(a) = f(b)$, então existe ponto $c \in (a, b)$ tal que $f'(c) = 0$.

(Esse é um caso particular do terema do valor médio.)

Em outras palavras, esse teorema garante que sempre que uma função tiver valores iguais em dois pontos distintos, há entre eles pelo menos um ponto onde a derivada é nula. Vamos aplicar esse teorema sucessivamente em $g$. Como $g$ é continuamente $n + 1$ vezes diferenciável com pelo menos $n + 2$ zeros em $[a, b]$, usando o teorema de Rolle vemos que $g'$ é $n$ vezes diferenciável com $n + 1$ zeros em $(a, b)$. Agora, aplicando o teorema em $g'$, concluímos que $g''$ é $n - 1$ vezes diferenciável com $n$ zeros em $(a, b)$. Esse jogo pode continuar até que concluímos que existe $\xi \in (a, b)$ , tal que $g^{(n + 1)}(\xi) = 0$.

Por outro lado, podemos partir da definição de $g$ e ver que
$$
\begin{split}
g^{(n + 1)}(z) &= E(x) \omega^{(n + 1)}(z) - E^{(n + 1)}(z) \omega(x) \\
&= E(x)(n + 1)! - f^{(n + 1)}(z) \omega(x).
\end{split}
$$
Portanto, descobrimos que para $z = \xi$
$$
E(x)(n + 1)! = f^{(n + 1)}(\xi) \omega(x).
$$
Nessa última passagem usamos que $p^{(n + 1)} \equiv 0$, já que $p$ é polinômio de grau até $n$.

Ou seja, obtemos
$$
E(x) = f(x) - p(x) = \frac{f^{(n + 1)}(\xi)}{(n + 1)!} \omega(x),\ x \neq x_i, i = 1, 2, \ldots, n,
$$
em que $\omega(x) = (x - x_0)(x - x_1) \ldots (x - x_n)$ e $\xi \in (a, b)$ (lembre que $\xi$ muda para cada $x$).

### Exemplo

Considere a função $f(x) = \sqrt{x}$, tabelada em $1, 2, 4$ e o seu polinômio interpolador.


In [ ]:
xs = [1, 2, 4]
ys = sqrt.(xs)
A = [ones(length(xs)) xs xs.^2]
c = A \ ys
p(x) = c[1] + c[2]*x + c[3]*x^2
xxs = range(1, 4, length=200)
plot(xxs, sqrt.(xxs), lw=2, label = L"\sqrt{x}")
plot!(xxs, p.(xxs), lw=2, label = "p(x)", legend = :topleft)

Como você pode ver, a aproximação parece muito boa. Mas será que conseguimos quantificar isso? O resultado anterior nos permite estimar o máximo erro no intervalo. 
$$E_{\max} = \max \left|  \frac{f'''(\xi)}{3!} \omega(x) \right| \leq  \frac{\max_{x \in [1, 4]}f'''(x)|}{6} \max_{x \in [1, 4]} |\omega(x)|.$$
Vamos estimar o máximo acima.

Inicialmente,
$$f'''(x) = \frac{3}{8} \frac{1}{\sqrt{x^5}}.$$
Como essa função é positiva decrescente, o máximo de seu móudlo é atingido em $x = 1$, sendo $3 /8$.
Já o máximo de $| \omega(x) |$ exige um pouco mais de cuidado. Primeiro note que fora dos pontos onde essa função se anula, que são 1, 2 e 4, ela é diferenciável. Assim, para estudar o seu máximo em $[1, 4]$, devemos considerar os dois intervalos $[1, 2]$ e $[2, 4]$. O ponto de máximo terá que ser:

* Ou um ponto do interior desses intervalos, onde a derivada se anule.
* Ou um dos extremos dos intervalos, mas lembrando que aí a função vale $0$ e portanto eles podem se desconsiderados. São o mínimo valor possível para a função.

A derivada de $\omega$ é 
$$\omega'(x) = 3 x^2 - 14 x + 14.$$ 
Assim, seus pontos de derivada nula são 
$$x = \frac{7 \pm \sqrt{7}}{3}.$$ 
Os dois pontos estão no intervalo $[1, 4]$ e precisam ser considerados. 

In [ ]:
x1, x2 = (7 + √7) / 3, (7 - √7) / 3

In [ ]:
ω(x) = (x - 1)*(x - 2)*(x - 4)
x1 = (7 + √7) / 3
x2 = (7 - √7) / 3
abs(ω(x1)), abs(ω(x2))

Ou seja, o máximo está na primeira raiz e é menor ou igual a $2.11262$. Concluimos que o módulo do erro máximo está limitado por
$$E{\max} \leq \frac{3/8 \cdot 2.11262}{6}.$$

In [ ]:
(3/8 * 2.11262) / 6

Concluímos que $E_{\max} \leq 0.13204$. Isso explica porque a figura parece tão boa.

Notem que uma das dificuldades de estimar o erro máximo é encontrar os máximos de funções. Mesmo se essas forem polinômios isso pode ser uma tarefa difícil se os graus forem altos ou se $f$ tiver. Além disso, sempre iremos precisar estimar o máximo de $f^{(n + 1)}(x)$, pois não sabemos onde está $\xi$. Já o máximo de $|\omega(x)|$ só é necessário se queremos um único número que funcione para todo o intervalo. Se estivermos interessados em um único valor temos uma tarefa mais simples para fazer.

#### Exercício 

Com que grau de precisão conseguimos aproximar $\sqrt{115}$ usando interpolação quadrática sobre os pontos $100$, $121$ e $144$?

Para resolver isso começamos como anteriormente e concluímos que, como,
$$f'''(x) = \frac{3}{8} \frac{1}{\sqrt{x^5}}.$$
É decrescente e positiva, o máximo de seu módulo no intervalo [$100, 144]$ é o seu valor calculado em 100, ou seja:
$$\frac{3}{8} \frac{1}{10^5} = 3.75 \cdot 10^{-6}.$$


Concluimos que
$$|E(115)| \leq \frac{3.75\cdot 10^{-6} |ω(115)|}{6},$$
com

In [ ]:
ω(x) = (x - 100)*(x - 121)*(x - 144)
3.75e-6 * ω(115) / 6


E assim $|E(115)| \leq 1.6313 \cdot 10^{-3}.$

# Reduzindo o erro

Até esse momento parece que aumentar o número de pontos de interpolação deve levar a aproximações cada vez mais precisas da função desejada. Será que isso é mesmo verdade.

Para isso vamos usar polinômios interpoladores ara aproximar a função que descreve a distribuição normal e ver o que ocorre.

In [ ]:
# Função de Runge
f(x) = 1.0 / (1.0 + 25.0*x^2)
l, r = -1, 1

# Calcula interpolação
function interpola(f, l, r, npoints)
    xs = range(l, r, length = npoints)
    exps = 0:length(xs) - 1
    V = [x^i for x in xs, i in exps]
    c = V \ f.(xs)
    p(x) = sum(c[i + 1]*x^i for i in exps)
    return xs, p
end

# Calcula dois polinômios interpoladores
xs1, pp1 = interpola(f, l, r, 5)
xs2, pp2 = interpola(f, l, r, 11)

# Apresenta os gráficos
xs = range(l, r, length=200)
plot(xs, f.(xs), lw=2, label = L"f")
scatter!(xs1, f.(xs1), label = "", color = :green)
plot!(xs, pp1.(xs), lw=2, label = "grau $(length(xs1))", color = :green)
scatter!(xs2, f.(xs2), label = "", color = :red)
plot!(xs, pp2.(xs), lw=2, label = "grau $(length(xs2))", color = :red, legend = :top)

Como podemos ver, ao aumentar o grau conseguimos diminuir o erro no meio, mas aparece mais oscilação ns extremos do intervalo. Brinque de aumentar o grau para ver o que ocorre. Você verá que essa oscilação vai ficando cada vez mais forte. Esse fenômeno é conhecido como fenômeno de Runge.

Esse fato coloca em xeque se devemos mesmo usar essa estratégia de aumentar o grau. Será que há outra forma de diminuir o erro?

Na literatura matemática há algumas soluções possíveis. A primeira está relacionada com escolher pontos especiais para fazer a interpolação, ao invés de usar a solução simples de pontos igualmente espaçados. Esses pontos de interpolação são zeros dos polinômios de Chebyschev. Não vamos nos estender nesse assunto nesse curso, mas é importante você saber que essa possibilidade existe.

Mas essa solução ainda tem uma dificuldade. Por vezes não temos como escolher os pontos de interpolação. Imagine a situação que a tabela já foi obtida por experimentos caros e os pontos de interpolação são equidistantes. Não é possível trocar os pontos, ou isso seria muito custoso. O que fazer nesse caso?

Uma opção é separa os pontos em subintervalos menores com extremos em comum e em cada um deles definir um polinômio interpolador. Como os extremos são comuns a função definida é pelo menos contínua. Vejamos o que seria obtida no caso anterior se usássemos subintervalos de apenas dois pontos, gerando uma função _linear por partes_.









In [ ]:
# Apresenta os gráficos
plot(xs, f.(xs), lw=2, label = L"f")
scatter!(xs2, f.(xs2), label = "", color = :red)
plot!(xs2, f.(xs2), lw=2, label = "grau 1 por partes", color = :red, legend = :topleft)

Note que a função obtida é contínua e o fenômeno de oscilação, naturalmente mudou. Por outro lado, o comportamento no bico, parece que ficou ruim. O que podemos fazer? Uma opção seria usar polinômios de grau 2 para pegar a "curvatura", agora agrupando os pontos de interpolação de três em três. Vamos tentar fazer isso de forma curta e rápida abaixo.

In [ ]:
pols = [interpola(f, xs2[i], xs2[i + 2], 3)[2] for i = 1:2:length(xs2) - 2]
plot(xs, f.(xs), lw=2, label = L"f")
scatter!(xs2, f.(xs2), label = "", color = :red)
for i = 1:2:length(xs2) - 2
    pp2 = pols[div(i, 2) + 1]
    l, r = xs2[i], xs2[i + 2]
    xssub = range(l, r, length=50)
    plot!(xssub, pp2.(xssub), color = :red, label = "")
end
title!("Polinômio de grau 2 por partes")

Vamos aumentar o grau de um a mais, para isso precisamos criar outro vetor de pontos de interpolação.

In [ ]:
xsn = range(-1, 1, length=13)
grau = 3
pols = [interpola(f, xsn[i], xsn[i + grau], grau + 1) for i = 1:grau:length(xsn) - grau]
plot(xs, f.(xs), lw=2, label = L"f")
scatter!(xsn, f.(xsn), label = "", color = :red)
for i = 1:grau:length(xsn) - grau
    int, pn = pols[div(i, grau) + 1]
    xssub = range(int[1], int[end], length=50)
    plot!(xssub, pn.(xssub), color = :red, label = "")
end
title!("Polinômio de grau $grau por partes")

O erro diminui no início da subida, mas de novo não ficou tão bom no bico. Isso ocorreu, entre outros motivos, porque o ponto do meio é justamente um ponto de mudança de polinômio. Então o polinômio da esquerda e da direita é influenciado pela mudança de concavidade que ocorre aí. 

De qualquer forma, demos uma nova ideia que é a ideia de _interpolação por partes_. Quando possuirmos muitos pontos de interpolação $x_0 < x_1 < x_2 < \ldots < x_n$, pode ser mais eficiente subdividi-los em subintervalos com extremos coincidentes $I_j,\ j = 1, \ldots, k$ e fazer a interpolação separada em cada subintervalo. Se $p_j$ é o polinômio que interpola no intervalo $I_j$, definimos a função interpoladora:
$$s(x) := p_j(x),\ \text{se $x \in I_j$}.$$
Note que, não há problemas nos extremos que pertencem a dois intervalos. Eles são pontos de interpolação então nos dois intervalos os polinômios interpoladores valem o mesmo. Essa forma de interpolação é conhecida com interpolação por partes $m$, onde $m$ é o grau dos polinômios interpoladores em cada subintervalo. No caso de $m = 1$ chamamos esse tipo de interpolação de interpolação por splines lineares.

Mas como fica o erro nesse caso? O erro de interpolação é o erro que ocorre em cada intervalo. Vamos analisar o caso linear. Lembrando o que já vimos o erro máximo em cada subintervalod é limitado por 
$$\max_{x \in I_j} | E(x) | \leq \frac{1}{2} \max_{x \in I_j} | f''(x) | \max_{x \in I_j} |(x - x_{j - 1})(x - x_j)|.$$
Nesse caso é fácil calcular o máximo de $(x - x_{j - 1})(x - x_j)$. Ele é $\frac{1}{4} h^2$, em que $h_j = x_j - x_{j - 1}$. 

Ainda, se quiséssemos achar um limitante para o erro em todo o intervalo original $[x_0, x_n]$, iríamos precisar estimar $\max_{x \in [x_0, x_n]} | f''(x)|$, vamos supor que conseguimos fazer isso e mostrar que ele é menor que um valor $M_2$. Como todo subintervalo $I_j \subset [x_0, x_n]$ concluímos que, para $h = \max_{j} h_j$,
$$\max_{x \in [x_0, x_n]} | E(x) | \leq \frac{M_2 h^2}{8}.$$

O que isso nos ensina? Isso nos ensina que podemos aproximar a função _tão bem quando desejemos_ com splines lineares, para isso basta subdividir o intervalo original em intervalos pequenos o suficiente e que a função tenha derivada segunda limitada no intervalo total. 

Ou seja, não há necessidade de se aumentar o grau do polinômio interpolador! Podemos ficar com interpoladores lineares, se pudermos usar muitos deles. De fato, a maioria das bibliotecas que apresentam gráficos de função apresentam o gráfico de uma aproximação splines lineares. É isso que faz o `Plots.jl` e é por isso que precisamos subdividir o intervalo onde a função será apresentada em muitos pontos.

Note que de posse de um resultado assim podemos resolver alguns problemas interessantes. Apresento dois exemplos a seguir.

#### Exercício

Em quantos pontos uniformemente espaçados no intervalo $[0, 2]$ devemos tabelar a função $f(x) = (2x + 1) / (x - 3)$ de maneira a garantir que o erro máximo atingido por uma interpolação por splines lineares é $10^{-4}$?

Para isso, precisamos calcular o máximo de $|f''|$:
$$M_2 = \max_{x \in [0, 2]}\left| \frac{14}{(x - 3)^3} \right| = 14.$$
Deste modo, queremos
$$| E(x) | \leq \frac{14 h^2}{8} = \frac{7h^2}{4} \leq 10^{- 4}.$$
Isso ocorre se
$$h \leq \frac{2}{\sqrt{7}} 10^{-2}.$$
Lembrando que $h = 2 / n$, isso equivale a
$$n \geq 100 \sqrt{7} \approx 264.5751.$$
Ou seja precisamos tomar $n \geq 265$ intervalos e, assim, precisamos de 266 pontos tabelados.

#### Exercício

Quantos pontos são necessários para tabelar a função cosseno para que sua aproximação por splines lineares resulte em erro menor que $10^{-4}$?

Neste caso, começamos por observar que basta tabelar a função no intervalo 
$[0, \pi/2]$. Se argumento não pertencer a esse intervalo, ele pode ser reduzido a um
valor aí. Com isso em mente, precisamos calcular o máximo do módulo da segunda
derivada do cosseno no intervalo. Mas essa segunda derivada é menos cosseno e assim o máximo do módulo é 1. Agora podemos aplicar a fórmula.
$$\frac{h^2}{8} \leq 10^{-4} \iff h \leq 2\sqrt{2}10^{-2}.$$
Mais uma vez, usando que $h = \pi / (2n)$ concluimos que
$$n \geq \frac{\pi}{4\sqrt{2}} 10^2 \approx 55.5.$$
Assim são necessários 56 subintervalos e, então, 57 pontos.

# Splines cúbicas

Uma das desvantagens das splines lineares, ou mesmo das splines de grau mais alto é que não conseguimos garantir que elas sejam funções diferenciáveis. Vimos isso de forma bem radical no exemplo de Runge, considerando o ponto do meio.

Mas há uma forma de lidar com isso. Podemos fazer a interpolação a cada dois pontos, como fazemos com splines lineares, mas de forma a garantir que a função interpolante $s$ seja tal que:

* $s$ é duas vezes continualmente diferenciável ($\mathcal{C}^2$).
* $s$ restrita a $[x_{j - 1}, x_j]$ é um polinômio de grau até 3.
* $s(x_j) = y_j$, para $j = 0, 1, 2, \ldots, n$.

Como isso é possível? Basta pensar um pouco. Os dois graus extra de liberdade do polinômio cúbico são usados para garantir que as derivadas primeiras e segundas dos polinômios vizinhos sejam iguais nos extremos. Como esperado isso tudo pode se feito com sistemas lineares, já que os polinômios e suas derivadas são lineares nos coeficientes usados para definí-los.

Note, porém que como os dois extremos do intervalos não possuem polinômios anteriores ou posteriores, as suas derivadas ficam livres. Para isso é necessário adicionar novas condições. Uma condição típica é pedir que essas derivadas sejam nulas ou iguais a valores conhecidos das derivadas das funções que se quer interpolar.